In [1]:
import torch
from transformers import BartForConditionalGeneration
import pandas as pd 
from datasets import *

from sklearn.model_selection import train_test_split

/Users/kayhan/opt/anaconda3/envs/pendulum/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-06 00:06:15.832152: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Experiment notes


- BART encoder input size is too short 
- 1 char is approx 4 tokens
- our input and output sizes are 2500 and 150 tokens


accidentally trained the model without stripping the original part. 

In [2]:
rmv_str = 'use the following opinions of the restaurant reviews to create a 50 word summary: \n'

In [6]:
df = pd.read_csv('../labeling jobs/data_to_label.csv')
data = df[['name','review','summary']]
data['review'] = data['review'].apply(lambda x : x.replace(rmv_str,''))

/var/folders/vy/y5363_9d6f54v8kgq1spp7n80000gn/T/ipykernel_56366/3147033629.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(lambda x : x.replace(rmv_str,''))


In [7]:
data_train, data_test = train_test_split(data,test_size=0.2,random_state=0)

In [8]:
train_dataset = Dataset.from_pandas(data_train,split='train')
test_dataset = Dataset.from_pandas(data_test,split='test')
ds_dict = {'train':train_dataset,'test':test_dataset}
dataset = DatasetDict(ds_dict)
dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'review', 'summary', '__index_level_0__'],
        num_rows: 275
    })
    test: Dataset({
        features: ['name', 'review', 'summary', '__index_level_0__'],
        num_rows: 69
    })
})

In [13]:
dataset['test'][0]

{'name': 'Hu Tong Jian Bing',
 'review': " Love that this is an addition to the Dogpatch area! I remember when this first came to SF I waited 30 min in line at a SPARK food festival to get my hands on one. So nice to walk in and order in an actual brick and mortar joint! This isn't like the traditional ones I've had in China but they're still tasty! The only downsides are as others have mentioned:1. The sauce isn't as spicy as flavorful as I had hoped. Even with the addition of the $1 picked veggie it still lacked a kick. I'm going to order extra spicy with extra sauce next time.2. The outside was a bit undercooked and doughy. Still good but it wasn't as well done as it could be. Still going to frequent this joint though.,I got the roast duck jian bing and the mango sago. Both were really good and reminded me of authentic jian bing!,Happy Cow Creamery & Tea is an Asian dessert spot in the Dogpatch. I've recently just noticed it and was surprised to find out that they house a permanent 

In [14]:
import evaluate

rouge = evaluate.load("rouge")

In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from transformers import LEDForConditionalGeneration, LEDTokenizer


# checkpoint = 'allenai/led-base-16384'
# checkpoint = 'allenai/PRIMERA'
checkpoint = 'facebook/bart-base'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
# tokenizer = LEDTokenizer.from_pretrained(checkpoint)
# model = LEDForConditionalGeneration.from_pretrained(checkpoint)
# model = BartForConditionalGeneration.from_pretrained(checkpoint)


# from transformers import (

#     AutoTokenizer,

#     LEDConfig,

#     LEDForConditionalGeneration,

# )

# tokenizer = AutoTokenizer.from_pretrained('allenai/PRIMERA')

# config=LEDConfig.from_pretrained('allenai/PRIMERA')

# model = LEDForConditionalGeneration.from_pretrained('allenai/PRIMERA')


In [16]:
tokenizer.model_max_length

1024

In [17]:
df['review'].apply(len).median() / 4

2348.25

In [18]:
df['summary'].apply(len).median() / 4

128.5

In [19]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["review"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=150, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.99ba/s]


In [21]:
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [22]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="generative_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, review, __index_level_0__, name. If summary, review, __index_level_0__, name are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/Users/kayhan/opt/anaconda3/envs/pendulum/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 275
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 72
  Number of trainable parameters = 139420416
You're using a BartTokenizerFast tokenizer

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.190973,0.200600,0.095400,0.167500,0.167400,20.000000
2,No log,2.107524,0.202700,0.096400,0.170000,0.169800,20.000000
3,No log,2.074252,0.201700,0.094800,0.168900,0.168700,20.000000
4,No log,2.072805,0.203600,0.096400,0.171100,0.171100,20.000000


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, review, __index_level_0__, name. If summary, review, __index_level_0__, name are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 16
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate co

TrainOutput(global_step=72, training_loss=2.5587957170274525, metrics={'train_runtime': 4865.2319, 'train_samples_per_second': 0.226, 'train_steps_per_second': 0.015, 'total_flos': 670710104064000.0, 'train_loss': 2.5587957170274525, 'epoch': 4.0})

In [20]:
tuned_model = trainer.model

In [54]:
input_ = dataset['test']['review'][1]

In [55]:
encoded = tokenizer.encode(input_,return_tensors='pt',max_length=1024)

In [64]:
tuned_model.save_pretrained('yelpGPTv1')

Configuration saved in yelpGPTv1/config.json
Configuration saved in yelpGPTv1/generation_config.json
Model weights saved in yelpGPTv1/pytorch_model.bin


In [56]:
output = tuned_model.generate(input_ids=encoded,max_length=150)

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



In [50]:
tokenizer.decode(output[0])

'</s><s>Happy Cow Creamery & Tea is an Asian dessert spot in the Dogpatch area that serves Chinese street food. The service is friendly and helpful, and the food is delicious. However, some reviewers found the Chinese crepes lacking a kick, and some found the pork jian bing to be undercooked and doughy. Overall, the restaurant is a must-visit spot in San Francisco.</s>'

In [57]:
tokenizer.decode(output[0])

'</s><s>Alnico is a Vietnamese restaurant in San Francisco that serves delicious Vietnamese coffee with Ube Foam. The food and drinks are delicious, but the price is reasonable for everything that comes with the ube flavored stuff. The restaurant is located on Hill St. and has outdoor seating available and doggo friendly seating. The service is good and the food is good. However, the price can be high for a weekend brunch.</s>'

In [63]:
 dataset['test']['review'][2]

'use the following opinions of the restaurant reviews to create a 50 word summary: \n The very definition of a hidden gem.I found this place on Yelp and my friend made a reservation for 6 people (they DID require a "deposit")This place has limited indoor seating due to the super cool 2 story layout. Kinda a small dining room on main level, then a balcony of seats on floor 2 that surrounds the dining room on 2 sides. ¬†Our table of 6 was all by itself on the right upper level- so perfect for us to all catch up with each other.Our FANTASTIC waiter Frederick was friendly and attentive, I wanted a lot of ice, he brought us 2 pitchers of ice! We ordered drinks (note that due to them only have a wine/beer license, their "cocktails" are made with low-proof rum or vodka.)We ordered some fried okra and some corn muffins to start and share. ¬†The okra comes with a snappy dipping sauce. (Note the corn muffins are pretty small).I ordered the fried chicken (leg and thigh) with Jambalaya and the col

In [25]:
test = tokenized_dataset['test']['input_ids'][0]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('allenai/PRIMERA')
tokenizer.model_max_length

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('allenai/led-large-16384')
tokenizer.model_max_length

In [ ]:

tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
tokenizer.model_max_length

In [ ]:

tokenizer = AutoTokenizer.from_pretrained('google/pegasus-large')
# model = GPT2Model.from_pretrained('gpt2')
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)
tokenizer.model_max_length